# Neural network to approximate Runge's function

In [1]:
# Imports
import autograd.numpy as np  # We need to use this numpy wrapper to make automatic differentiation work later
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import (
    PolynomialFeatures,
)

from functions import runge, OLS_parameters
from functions import ReLU, ReLU_der, sigmoid, sigmoid_der, softmax, softmax_vec, mse_der
from functions import create_layers, feed_forward_batch, feed_forward_saver, backpropagation, train_network, backpropagation_cross

In [2]:
np.random.seed(42)
n = 1000
# data set from project 1
x = np.linspace(-1,1, n).reshape(-1,1)
y = runge(x) + 0.1*np.random.normal(0,1, x.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_offset = y_train.mean()

poly = PolynomialFeatures(degree=10)
X_train = poly.fit_transform(x_train)
X_test = poly.fit_transform(x_test)
# scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# OLS
beta = OLS_parameters(X_train_s, y_train)
y_test_tilde = X_test_s @ beta + y_offset
y_train_tilde = X_train_s @ beta + y_offset

In [3]:
# def func(x):
#     return x

# def func_der(x):
#     return np.array([1]*len(x))

In [ ]:
layer_output_sizes = [50, 1]
activation_funcs = [sigmoid, ReLU]
activation_ders = [sigmoid_der, ReLU_der]

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
inputs = x_train_s
targets = y_train + y_offset

network_input_size = inputs.shape[1]
layers = create_layers(network_input_size, layer_output_sizes)
train_network(inputs, layers, activation_funcs, activation_ders, targets, mse_der, learning_rate=0.01, epochs=100, minibatch_size=15)
# train_network_cross(inputs, layers, activation_funcs, activation_ders, targets, learning_rate=0.01, epochs=100, minibatch_size=15)



Backpropagation - Layer: 1
Last layer
dC_dW shape: (750, 15)
dC_db shape: (15, 1)
dC_dz shape: (15, 1)
dC_da shape: (15, 1)
Backpropagation - Layer: 0
No last layer
W shape: (50, 1)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 15 is different from 1)

In [ ]:
def accuracy(predictions, targets):
    one_hot_predictions = np.zeros(predictions.shape)

    for i, prediction in enumerate(predictions):
        one_hot_predictions[i, np.argmax(prediction)] = 1
    return accuracy_score(one_hot_predictions, targets)

In [ ]:
targets = y_test + y_offset

predictions = feed_forward_batch(X_test_s, layers, activation_funcs)
print(accuracy(predictions, targets))